In [1]:
import csv
import numpy as np
import pandas as pd
import glob
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
import seaborn as sns
from tqdm import tqdm
import math
import pickle
from scipy import stats

In [5]:
df = pd.read_csv('../../../DataSx_tables/230325_Dataset3_mastertable.csv')
#df = pd.read_csv('./Dataset1_mastertable.csv')

df

,name,dna_seq,log10_K50_t,log10_K50_t_95CI_high,log10_K50_t_95CI_low,log10_K50_t_95CI,fitting_error_t,log10_K50unfolded_t,deltaG_t,deltaG_t_95CI_high,...,deltaG_c,deltaG_c_95CI_high,deltaG_c_95CI_low,deltaG_c_95CI,deltaG,deltaG_95CI_high,deltaG_95CI_low,deltaG_95CI,log10_K50_trypsin_ML,log10_K50_chymotrypsin_ML
0,1A32.pdb,TCTGCGGGTGGCTCTCCAGAAGTTCAGATTGCGATCCTGACCGAAC...,0.240062,0.259872,0.218340,0.041532,0.069827,-1.781164,2.696992,2.723847,...,2.620790,2.653094,2.596824,0.056270,2.674476,2.723361,2.644279,0.079082,0.2400615540197126,-0.7641209993449366
1,1A32.pdb_A45D,TCTGCGGGTGGTTCTCCGGAAGTTCAGATCGCTATTCTGACCGAAC...,0.245272,0.271181,0.237714,0.033467,0.046275,-1.728073,2.632435,2.667598,...,2.487671,2.527694,2.473564,0.054130,2.558410,2.595347,2.516665,0.078682,0.2452717737233243,-0.7123364923356303
2,1A32.pdb_A45E,TCTGCGGGTGGTTCTCCGGAAGTTCAGATCGCTATTCTGACCGAAC...,0.411134,0.421462,0.402358,0.019104,0.045609,-1.726210,2.854924,2.868932,...,2.782184,2.807850,2.749225,0.058626,2.830464,2.879223,2.791301,0.087922,0.4111336742937279,-0.545467769242733
3,1A32.pdb_A45F,TCCGCTGGCGGTTCTCCGGAAGTTCAGATCGCTATTCTGACCGAAC...,0.038186,0.051789,-0.001474,0.053263,0.072121,-1.793375,2.439442,2.457941,...,2.159766,2.200242,2.082575,0.117667,2.280177,2.351299,2.212888,0.138411,0.0381855701102482,-1.1379101516061252
4,1A32.pdb_A45G,TCTGCTGGTGGCTCTCCGGAAGTTCAGATCGCTATTCTGACCGAAC...,-0.017184,0.013252,-0.031242,0.044494,0.052017,-1.765509,2.326206,2.367694,...,2.292774,2.313889,2.260935,0.052955,2.349356,2.404530,2.269762,0.134769,-0.0171837522091489,-0.9757493546746832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1841280,5GHB.pdb_I68C,TCCGCGGGTGGTCACATCAACCTGAAAGTTGCGGGTCAGGACGGTT...,-0.659227,-0.592633,-0.713220,0.120587,0.063869,-1.675883,1.299455,1.397173,...,0.343328,0.561563,0.159775,0.401787,0.799442,0.896299,0.721957,0.174342,-0.6592273716286323,-2.32007634084457
1841281,5GHB.pdb_D69C,TCTGCTGGCGGTCACATCAACCTGAAAGTTGCGGGTCAGGACGGTT...,1.526104,1.553159,1.500452,0.052707,0.064731,-1.690861,4.361019,4.401617,...,4.077375,4.105002,4.025692,0.079311,4.309125,4.351921,4.272543,0.079379,1.5261042575493224,0.2675031199165091
1841282,5GHB.pdb_V70C,TCTGCTGGCGGTCACATTAACCTGAAAGTTGCGGGTCAGGACGGCT...,0.189385,0.213863,0.152034,0.061828,0.102480,-1.678700,2.489757,2.523049,...,2.387179,2.601802,2.213221,0.388582,2.555101,2.643876,2.396862,0.247014,0.1893847673240727,-0.9724388178041864
1841283,5GHB.pdb_F71C,TCCGCTGGCGGCCATATCAACCTGAAAGTTGCGGGTCAGGACGGTT...,1.406698,1.424117,1.375079,0.049039,0.073401,-1.670525,4.157925,4.183294,...,3.800988,3.817341,3.760982,0.056359,4.049197,4.084906,4.015010,0.069896,1.4066979508059534,0.0777885370976122


In [6]:
mut_list = list('QENHDRKTSAGMLVIWYFPC') + ['insG','insA','del']

In [7]:
# make sub-dataframe using name of domain"


def make_subdf(name):
    subdf = df[(df['name'].str.startswith(name))]
    
    subdf_wt = subdf[subdf['name']==name]
    subdf_wt = pd.concat([subdf_wt,subdf[subdf['name'].str.contains('wt')]])
    
    subdf_nowt = subdf[subdf['name']!=name]
    subdf_nowt = subdf_nowt[subdf_nowt['name']!=name+'.pdb']
    subdf_nowt = subdf_nowt[~subdf_nowt['name'].str.contains('wt')]
    
    return subdf_wt,subdf_nowt

In [8]:
# calculate distance of WT times and fitting line

from scipy.odr import Model, RealData, ODR

def f(B, x):
    '''Linear function y = m*x + b'''
    return B[0]*x + B[1]
linear = Model(f)

def distance_line_dot(a,b,x0,y0):
    return np.abs(a*x0-y0+b)/((a**2+1)**(1/2))

In [9]:
# make array 23 (20 amino acids + Gly insert + Ala insert + deletion) x amino acid length for K50, deltaG etc

def df2KTCarray(name,wtseq,subdf_wt,subdf_nowt):
    
    mut_list = list('QENHDRKTSAGMCLVIWYFP') + ['insG','insA','del']

    kT_array=np.resize(np.median(subdf_wt['log10_K50_t']), (len(wtseq),len(mut_list)))
    kC_array=np.resize(np.median(subdf_wt['log10_K50_c']), (len(wtseq),len(mut_list)))
    deltaG_t_array=np.resize(np.median(subdf_wt['deltaG_t']), (len(wtseq),len(mut_list)))
    deltaG_c_array=np.resize(np.median(subdf_wt['deltaG_c']), (len(wtseq),len(mut_list)))
    deltaG_array=np.resize(np.median(subdf_wt['deltaG']), (len(wtseq),len(mut_list)))
    
    NA_list , lowconf_list_t,lowconf_list_c,lowconf_list_dgt,lowconf_list_dgc,lowconf_list_dg = [],[],[],[],[],[]
    
    for i in range(len(wtseq)):
        for j, aa in enumerate(mut_list):
            #print(j,aa)
            try:
                if (wtseq[i] != aa) & (j<20):
                    kT_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['log10_K50_t'].values[0]
                    kC_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['log10_K50_c'].values[0]
                    deltaG_t_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['deltaG_t'].values[0]
                    deltaG_c_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['deltaG_c'].values[0]
                    deltaG_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['deltaG'].values[0]

                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['log10_K50_t_95CI'].values[0]>0.5:
                        lowconf_list_t.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['log10_K50_c_95CI'].values[0]>0.5:
                        lowconf_list_c.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['deltaG_t_95CI'].values[0]>0.5:
                        lowconf_list_dgt.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['deltaG_c_95CI'].values[0]>0.5:
                        lowconf_list_dgc.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa))['deltaG_95CI'].values[0]>0.5:
                        lowconf_list_dg.append(str(i)+'_'+str(j))

                elif (j==20)|(j==21):
                    #print(j,aa)
                    kT_array[i, j] = subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['log10_K50_t'].values[0]
                    kC_array[i, j] = subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['log10_K50_c'].values[0]
                    deltaG_t_array[i, j] = subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['deltaG_t'].values[0]
                    deltaG_c_array[i, j] = subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['deltaG_c'].values[0]
                    deltaG_array[i, j] = subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['deltaG'].values[0]
                    if subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['log10_K50_t_95CI'].values[0]>0.5:
                        lowconf_list_t.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['log10_K50_c_95CI'].values[0]>0.5:
                        lowconf_list_c.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['deltaG_t_95CI'].values[0]>0.5:
                        lowconf_list_dgt.append(str(i)+'_'+str(j))    
                    if subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['deltaG_c_95CI'].values[0]>0.5:
                        lowconf_list_dgc.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1))['deltaG_95CI'].values[0]>0.5:
                        lowconf_list_dg.append(str(i)+'_'+str(j))
                elif (j==22):
                    kT_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['log10_K50_t'].values[0]
                    kC_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['log10_K50_c'].values[0]
                    deltaG_t_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['deltaG_t'].values[0]
                    deltaG_c_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['deltaG_c'].values[0]
                    deltaG_array[i, j] = subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['deltaG'].values[0]

                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['log10_K50_t_95CI'].values[0]>0.5:
                        lowconf_list_t.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['log10_K50_c_95CI'].values[0]>0.5:
                        lowconf_list_c.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['deltaG_t_95CI'].values[0]>0.5:
                        lowconf_list_dgt.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['deltaG_c_95CI'].values[0]>0.5:
                        lowconf_list_dgc.append(str(i)+'_'+str(j))
                    if subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1))['deltaG_95CI'].values[0]>0.5:
                        lowconf_list_dg.append(str(i)+'_'+str(j))

            except:
                NA_list.append(str(i)+'_'+str(j))
                continue

    wt_indices=np.array([mut_list.index(aa) for aa in wtseq])

    return kT_array,kC_array,deltaG_t_array,deltaG_c_array,deltaG_array,wt_indices,NA_list,lowconf_list_t,lowconf_list_c,lowconf_list_dgt,lowconf_list_dgc,lowconf_list_dg

In [10]:
# identify sites that should be removed for training data

def cite_remover(kT_array,kC_array,kT_wt,kC_wt,wtseq):
    aas = list('QENHDRKTSAGMCLVIWYFP')
    outtable = np.concatenate([kT_array[:,:20],kC_array[:,:20],kT_array[:,:20]-kT_wt,kC_array[:,:20]-kC_wt],axis=1)
    #print(np.shape(outtable))
    outtable=pd.DataFrame(outtable)
    outtable.columns = ['K50_t_%s' % aa for aa in aas] + ['K50_c_%s' % aa for aa in aas] + ['dK50_t_%s' % aa for aa in aas] + ['dK50_c_%s' % aa for aa in aas]
    
    outtable['pos'] = range(1,len(wtseq)+1)
    
    
    ## identify sites where protease-target amino acid introduction destabilizes a domain
    
    KR_index = sorted([aas.index('R'), aas.index('K')])
    KRWYF_index = sorted([aas.index('R'), aas.index('K'), aas.index('W'), aas.index('Y'), aas.index('F')])
    WYF_index = sorted([aas.index('W'), aas.index('Y'), aas.index('F')])
    non_KRWYF_aas = 'QENDHTSAGMLVIP'
    non_WYFLM_aas = 'QENDHRKTSAGVIP'

    argsort_dt = np.argsort(outtable[['dK50_t_%s' % aa for aa in aas]].to_numpy(),axis=1)
    argsort_dc = np.argsort(outtable[['dK50_c_%s' % aa for aa in aas]].to_numpy(),axis=1)

    outtable['argsort_dK50_t'] = [x for x in argsort_dt]
    outtable['argsort_dK50_c'] = [x for x in argsort_dc]


    outtable['KR_min_t'] = [KR_index == sorted(x[0:2]) for x in outtable['argsort_dK50_t']  ]
    outtable['KRWYF_min_t'] = [KRWYF_index == sorted(x[0:5]) for x in outtable['argsort_dK50_t']  ]
    outtable['KR_or_KRWYF_min_t'] = [x or y for x,y in zip(outtable['KR_min_t'], outtable['KRWYF_min_t'])]
    outtable['WYF_min_c'] = [WYF_index == sorted(x[0:3]) for x in outtable['argsort_dK50_c']  ]

    outtable['non_KRWYF_dt_mean'] = np.average(outtable[['dK50_t_%s' % aa for aa in non_KRWYF_aas]].to_numpy(),axis=1)
    outtable['non_KRWYF_dt_std'] = np.std(outtable[['dK50_t_%s' % aa for aa in non_KRWYF_aas]].to_numpy(),axis=1)
    outtable['non_WYFLM_dc_mean'] = np.average(outtable[['dK50_c_%s' % aa for aa in non_WYFLM_aas]].to_numpy(),axis=1)
    outtable['non_WYFLM_dc_std'] = np.std(outtable[['dK50_c_%s' % aa for aa in non_WYFLM_aas]].to_numpy(),axis=1)

    outtable['KR_dt'] = np.average(outtable[['dK50_t_%s' % aa for aa in 'KR']].to_numpy(),axis=1)
    outtable['WYF_dc'] = np.average(outtable[['dK50_c_%s' % aa for aa in 'WYF']].to_numpy(),axis=1)

    outtable['KR_dt_over_std'] = outtable['KR_dt'] / outtable['non_KRWYF_dt_std']
    outtable['WYF_dc_over_std'] = outtable['WYF_dc'] / outtable['non_WYFLM_dc_std']

    outtable['KR_dt_over_mean'] = outtable['KR_dt'] / np.abs(outtable['non_KRWYF_dt_mean'])
    outtable['WYF_dc_over_mean'] = outtable['WYF_dc'] / np.abs(outtable['non_WYFLM_dc_mean'])
    
    table_destab = outtable.query("""non_WYFLM_dc_std < 0.25 & non_WYFLM_dc_mean > -0.3 & non_WYFLM_dc_mean < 0.4 & non_KRWYF_dt_std < 0.25 & non_KRWYF_dt_mean > -0.3 & non_KRWYF_dt_mean < 0.4 & KR_or_KRWYF_min_t == True & WYF_min_c == True""")


    ## identify sites where Cys introduction stablized a domain
    
    C_index = [aas.index('C')]
    non_C_aas = 'QENDHRKTSAGMLVIWYFP'
    outtable['C_max_t'] = [aas.index('C') == x[-1] for x in outtable['argsort_dK50_t']  ]
    outtable['C_max_c'] = [aas.index('C') == x[-1] for x in outtable['argsort_dK50_c']  ]
    outtable['non_C_dt_std'] = np.std(outtable[['dK50_t_%s' % aa for aa in non_C_aas]].to_numpy(),axis=1)
    outtable['non_C_dc_std'] = np.std(outtable[['dK50_c_%s' % aa for aa in non_C_aas]].to_numpy(),axis=1)
    outtable['non_C_dt_mean'] = np.mean(outtable[['dK50_t_%s' % aa for aa in non_C_aas]].to_numpy(),axis=1)
    outtable['non_C_dc_mean'] = np.mean(outtable[['dK50_c_%s' % aa for aa in non_C_aas]].to_numpy(),axis=1)

    outtable['C_dt_over_std'] = outtable['dK50_t_C'] / outtable['non_C_dt_std']
    outtable['C_dc_over_std'] = outtable['dK50_c_C'] / outtable['non_C_dc_std']

    outtable['C_dt_over_mean'] = [0 if y==0 else x/y for x,y in zip(outtable['dK50_t_C'],np.abs(outtable['non_C_dt_mean']))]
    outtable['C_dc_over_mean'] = [0 if y==0 else x/y for x,y in zip(outtable['dK50_c_C'],np.abs(outtable['non_C_dc_mean']))]

    table_cys_stab = outtable.query("""C_max_t==True & C_max_c==True & dK50_t_C > 0 & dK50_c_C > 0 & ((C_dt_over_mean>2)|(C_dt_over_std>2)) &((C_dc_over_mean>2)|(C_dc_over_std>2))""")
    
    return np.array(table_destab['pos']),np.array(table_cys_stab['pos'])

In [11]:
# collect basic information for filtering

def select_data_set(name,wtseq):
    
    subdf_wt,subdf_nowt=make_subdf(name)
    kT_array,kC_array,deltaG_t_array,deltaG_c_array,deltaG_array,wt_indices,NA_list , lowconf_list_t,lowconf_list_c,lowconf_list_dgt,lowconf_list_dgc,lowconf_list_dg = df2KTCarray(name,wtseq,subdf_wt,subdf_nowt) 

    
    # collect basic info for wt amino acid sequnece
    df_wt_median = pd.DataFrame()
    df_wt_median['log10_K50_t'] = [np.median(subdf_wt['log10_K50_t'])]
    df_wt_median['log10_K50_c'] = [np.median(subdf_wt['log10_K50_c'])]
    df_wt_median['deltaG_t'] = [np.median(subdf_wt['deltaG_t'])]
    df_wt_median['deltaG_c'] = [np.median(subdf_wt['deltaG_c'])]
    df_wt_median['deltaG'] = [np.median(subdf_wt['deltaG'])]
    df_wt_median['deltaG_t_95CI'] = [0.0]
    df_wt_median['deltaG_c_95CI'] = [0.0]
    
    
    # make dataframe including all mutants with or without wt data
    subdf_nowt2 = pd.DataFrame()
    subdf2 = pd.DataFrame()   
    for i in range(len(wtseq)):
        for j, aa in enumerate(mut_list):
            if (wtseq[i] != aa) & (j<20):
                subdf2 = subdf2.append(subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa)))
                subdf_nowt2 = subdf_nowt2.append(subdf_nowt.query('name == "%s_%s%s%s"' % (name,wtseq[i], i+1, aa)))
            elif (wtseq[i] == aa):
                df_wt_median['name'] = [name+'_'+wtseq[i]+str(i+1)+aa]
                subdf2 = subdf2.append(df_wt_median)

            elif (j==20)|(j==21):
                subdf_nowt2 = subdf_nowt2.append(subdf_nowt.query('name == "%s_%s%s"' % (name, aa,i+1)))
            elif (j==22):
                subdf_nowt2 = subdf_nowt2.append(subdf_nowt.query('name == "%s_%s%s%s"' % (name, aa,wtseq[i],i+1)))
            
            
    subdf = pd.concat([subdf_wt,subdf_nowt2])
    subdf_highconf = subdf[(subdf['deltaG_t_95CI']<=0.5*1.33)&(subdf['deltaG_c_95CI']<=0.5*1.33)]
 
    # if subdf doesn't include reliable data sufficiently, return zeros
    if (len(subdf_highconf)<10)|(subdf_highconf['deltaG_t'].isnull().values.sum()+subdf_highconf['deltaG_c'].isnull().values.sum() != 0)|(len(subdf_wt)==0):
        return np.zeros(20)
    
    # collect basic information for filtering
    if 'pross' in name:
        NA_ratio = (18*len(wtseq)+5-len(subdf[(~subdf['name'].str.endswith('C'))&(~subdf['name'].str.endswith('ins'))&(~subdf['name'].str.endswith('del'))]))/(18*len(wtseq)+5) 
    else:
        NA_ratio = (22*len(wtseq)+5-len(subdf[~subdf['name'].str.endswith('C')]))/(22*len(wtseq)+5) 
    
    raw_corr = stats.pearsonr(subdf[(subdf['log10_K50_t_95CI']<0.5)&(subdf['log10_K50_c_95CI']<0.5)]['log10_K50_t']
                              ,subdf[(subdf['log10_K50_t_95CI']<0.5)&(subdf['log10_K50_c_95CI']<0.5)]['log10_K50_c'])[0]
    dg_corr = stats.pearsonr(subdf_highconf['deltaG_t'],subdf_highconf['deltaG_c'])[0]
    
    width_KT = np.percentile(subdf_highconf['log10_K50_t'],97.5)-np.percentile(subdf_highconf['log10_K50_t'],2.5)
    width_KC = np.percentile(subdf_highconf['log10_K50_c'],97.5)-np.percentile(subdf_highconf['log10_K50_c'],2.5) # change 80CI to 95CI

    ktkc_data = RealData(subdf_highconf['deltaG_t'],subdf_highconf['deltaG_c'], sx=0.1, sy=0.1)
    od_ktkc = ODR(ktkc_data, linear, beta0=[1., 2.])
    out_kt_kc_high = od_ktkc.run()
    slope = out_kt_kc_high.beta[0]
    y_intercep = out_kt_kc_high.beta[1]
    
    wt_kt = np.median(subdf_wt['log10_K50_t'])
    wt_kc = np.median(subdf_wt['log10_K50_c'])
    
    wt_dg_std_max = max(np.std(subdf_wt['deltaG_t']),np.std(subdf_wt['deltaG_c']),np.std(subdf_wt['deltaG']))
    wt_k50_std_max = max(np.std(subdf_wt['log10_K50_t']),np.std(subdf_wt['log10_K50_c']))
    wt_dg_max = max((np.median(subdf_wt['deltaG_t']),np.median(subdf_wt['deltaG_c']),np.median(subdf_wt['deltaG'])))
    wt_k50_diff_max = max(max(subdf_wt['log10_K50_t'])-min(subdf_wt['log10_K50_t']),max(subdf_wt['log10_K50_c'])-min(subdf_wt['log10_K50_c']))
    wt_dgt_med = np.median(np.median(subdf_wt['deltaG_t']))
    wt_dgc_med = np.median(np.median(subdf_wt['deltaG_c']))
    wt_dg_med = np.median(np.median(subdf_wt['deltaG']))

    
    wt_d_list = []
    subdf_wt = subdf_wt.reset_index(drop=True)
    for i in range(len(subdf_wt)):
        wt_d_list.append(distance_line_dot(slope,y_intercep,subdf_wt['deltaG_t'],subdf_wt['deltaG_c']))
    wt_d_from_line = np.mean(wt_d_list)
    
    
    subdf_hydrophobic_replacement = subdf_nowt2[(subdf_nowt2['name'].str.endswith('F'))|(subdf_nowt2['name'].str.endswith('W'))|(subdf_nowt2['name'].str.endswith('Y'))|(subdf_nowt2['name'].str.endswith('M'))|(subdf_nowt2['name'].str.endswith('I'))|(subdf_nowt2['name'].str.endswith('L'))|(subdf_nowt2['name'].str.endswith('V'))]
    subdf_hydrophobic_replacement['pos'] = list([int(x.split('_')[-1][1:-1]) for x in subdf_hydrophobic_replacement['name']])
    
    ratio_pos_with_hydrophobic_stabilzing_muts = 0
    for pos in range(1,len(wtseq)+1):
        if len(subdf_hydrophobic_replacement[(subdf_hydrophobic_replacement['pos']==pos)]) - len(subdf_hydrophobic_replacement[(subdf_hydrophobic_replacement['pos']==pos)&(subdf_hydrophobic_replacement['deltaG']>np.median(subdf_wt['deltaG'])+0.25)]) <2:
            ratio_pos_with_hydrophobic_stabilzing_muts+=1
    ratio_pos_with_hydrophobic_stabilzing_muts = ratio_pos_with_hydrophobic_stabilzing_muts/len(wtseq)


    
    # calculate cleavage potential without global unfolding
    
    subdf_highconf_nowt = subdf_nowt2[(subdf_nowt2['deltaG_t_95CI']<=0.5*1.33)&(subdf_nowt2['deltaG_c_95CI']<=0.5*1.33)&(~subdf_nowt2['name'].str.contains('ins'))&(~subdf_nowt2['name'].str.contains('del'))]
    subdf_highconf_nowt['pos'] = list([int(x.split('_')[-1][1:-1]) for x in subdf_highconf_nowt['name']])
    subdf_highconf_nowt['mut_aa'] = list([(x.split('_')[-1][-1]) for x in subdf_highconf_nowt['name']])
    
    cut_potential_k50,cut_potential_dg = [],[]
    loop_site_list = []
    for i in range(1,len(wtseq)+1):
        subdf_pos = subdf_highconf_nowt[subdf_highconf_nowt['pos']==i]
        subdf_pos['deltaG_t'] = np.clip(subdf_pos['deltaG_t'],-1,5)
        subdf_pos['deltaG_c'] = np.clip(subdf_pos['deltaG_c'],-1,5)
        
        if (wtseq[i-1]=='K')|(wtseq[i-1]=='R'):
            #print(subdf_pos)
            mean_dK50_KR2others_T = np.mean(subdf_pos[(subdf_pos['mut_aa']!='K')&(subdf_pos['mut_aa']!='R')]['log10_K50_t'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='R')]['log10_K50_t']) #(subdf_pos['mut_aa']=='K')|
            mean_dK50_KR2others_C = np.mean(subdf_pos[(subdf_pos['mut_aa']!='K')&(subdf_pos['mut_aa']!='R')]['log10_K50_c'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='R')]['log10_K50_c']) #(subdf_pos['mut_aa']=='K')|
            mean_ddg_KR2others_T = np.mean(subdf_pos[(subdf_pos['mut_aa']!='K')&(subdf_pos['mut_aa']!='R')]['deltaG_t'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='R')]['deltaG_t']) #(subdf_pos['mut_aa']=='K')|
            mean_ddg_KR2others_C = np.mean(subdf_pos[(subdf_pos['mut_aa']!='K')&(subdf_pos['mut_aa']!='R')]['deltaG_c'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='R')]['deltaG_c'])
            
            if (mean_dK50_KR2others_T>0.5)|(mean_ddg_KR2others_T>0.5):
                cut_potential_k50.append(mean_dK50_KR2others_T-mean_dK50_KR2others_C)
                cut_potential_dg.append(mean_ddg_KR2others_T-mean_ddg_KR2others_C)
            else:            
                cut_potential_k50.append(0)
                cut_potential_dg.append(0)
            #print(mean_ddg_KR2others_T,mean_ddg_KR2others_C)
        elif (wtseq[i-1]=='F')|(wtseq[i-1]=='W')|(wtseq[i-1]=='Y')|(wtseq[i-1]=='M')|(wtseq[i-1]=='L'):

            mean_dK50_arom2others_T = np.mean(subdf_pos[(subdf_pos['mut_aa']!='F')&(subdf_pos['mut_aa']!='W')&(subdf_pos['mut_aa']!='Y')&(subdf_pos['mut_aa']!='L')&(subdf_pos['mut_aa']!='M')]['log10_K50_t'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='F')|(subdf_pos['mut_aa']=='W')|(subdf_pos['mut_aa']=='Y')]['log10_K50_t'])
            mean_dK50_arom2others_C = np.mean(subdf_pos[(subdf_pos['mut_aa']!='F')&(subdf_pos['mut_aa']!='W')&(subdf_pos['mut_aa']!='Y')&(subdf_pos['mut_aa']!='L')&(subdf_pos['mut_aa']!='M')]['log10_K50_c'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='F')|(subdf_pos['mut_aa']=='W')|(subdf_pos['mut_aa']=='Y')]['log10_K50_c'])
            mean_ddg_arom2others_T = np.mean(subdf_pos[(subdf_pos['mut_aa']!='F')&(subdf_pos['mut_aa']!='W')&(subdf_pos['mut_aa']!='Y')&(subdf_pos['mut_aa']!='L')&(subdf_pos['mut_aa']!='M')]['deltaG_t'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='F')|(subdf_pos['mut_aa']=='W')|(subdf_pos['mut_aa']=='Y')]['deltaG_t'])
            mean_ddg_arom2others_C = np.mean(subdf_pos[(subdf_pos['mut_aa']!='F')&(subdf_pos['mut_aa']!='W')&(subdf_pos['mut_aa']!='Y')&(subdf_pos['mut_aa']!='L')&(subdf_pos['mut_aa']!='M')]['deltaG_c'])-np.mean(subdf_pos[(subdf_pos['mut_aa']=='F')|(subdf_pos['mut_aa']=='W')|(subdf_pos['mut_aa']=='Y')]['deltaG_c'])
            
            if (mean_dK50_arom2others_C>0.5)|(mean_ddg_arom2others_C>0.5):
                cut_potential_k50.append(mean_dK50_arom2others_C-mean_dK50_arom2others_T)
                cut_potential_dg.append(mean_ddg_arom2others_C-mean_ddg_arom2others_T)
            else:            
                cut_potential_k50.append(0)
                cut_potential_dg.append(0)            

        else:
            cut_potential_k50.append(0)
            cut_potential_dg.append(0)
            

            
    cut_potential_k50_max = np.max(cut_potential_k50)
    cut_potential_dg_max = np.max(cut_potential_dg)
    
    destab_pos,cys_stab_pos = cite_remover(kT_array,kC_array,np.median(subdf_wt['log10_K50_t']),np.median(subdf_wt['log10_K50_c']),wtseq)

        
    return NA_ratio,raw_corr,dg_corr,slope,y_intercep,width_KT,width_KC,wt_dg_std_max,wt_k50_std_max,wt_dg_max,wt_dgt_med,wt_dgc_med,wt_dg_med,wt_kt,wt_kc,wt_k50_diff_max,wt_d_from_line,ratio_pos_with_hydrophobic_stabilzing_muts,cut_potential_k50_max,cut_potential_dg_max,destab_pos,cys_stab_pos

In [13]:
df_wts = pd.read_csv('../../Processed_K50_dG_datasets/Single_DMS_list.csv')
df_wts

,name,aa_seq,frac_NA,raw_corr,dg_corr,slope,y_intercept,width_KT,width_KC,wt_dg_std_max,...,wt_kt,wt_kc,wt_k50_diff_max,wt_d_from_line,frac_pos_with_hydrophobic_stabilzing_muts,cut_potential_k50_max,cut_potential_dg_max,WT_cluster,num_Cys,DMS_group
0,EA|run2_0325_0005.pdb,DEVTIHLGDKTIRVDGLDKELLEILKELARRGADEEELRKEIERWER,0.040423,0.882750,0.935385,0.952791,0.517992,2.053731,1.940963,0.302077,...,1.252483,1.175881,0.562823,0.140719,0.000000,0.000000,0.000000,EEHH,0,G0
1,EA|run3_0321_0005.pdb,HDVTIHAGDKTIHVHGASEEFLRIIEQAKRDGIHEEELLRLIEELVK,0.029836,0.748121,0.889368,0.972406,0.374393,1.701232,1.478200,0.082878,...,1.397474,1.563919,0.114670,0.120557,0.000000,0.000000,0.000000,EEHH,0,G0
2,EA|run3_0525_0006.pdb,SHFELRVGTITLHFDNISEELAEELEKLAKNGVHADEIRRWWEEWLK,0.044273,0.921225,0.937029,1.244125,-0.022625,1.682309,1.925590,0.119879,...,1.465792,1.175846,0.232397,0.119378,0.021277,0.000000,0.000000,EEHH,0,G0
3,EA|run3_1140_0005.pdb,FHVTIHVGDITFHIHGVSEEEVKKLEELVRRGADWREIKKTIEEIEK,0.036574,0.904704,0.887349,1.058797,0.717820,1.747861,1.808650,0.098991,...,1.539962,1.355212,0.169777,0.085652,0.000000,0.000000,0.000000,EEHH,0,G0
4,EA|run5_0050_0004.pdb,TEVDLHLGDITIKLKDVSEEIVKRAKELFKNGWSAEFVKKWIEEAVK,0.039461,0.964265,0.966722,1.078365,0.079215,2.248261,2.414449,0.099852,...,1.762778,1.338286,0.149830,0.094978,0.021277,0.000000,0.000000,EEHH,0,G0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,2J6K.pdb_pross9,VDYIVEYDYDAQHDDELTIRKGDIIKNVKKLEEEGWLEGELNGKRG...,0.001940,0.811083,0.856570,0.923100,0.602968,2.202907,1.738693,0.000000,...,1.091811,1.581180,0.000000,0.133743,0.070175,1.916736,2.247892,74,0,G11
979,r14_568_TrROS_Hall.pdb,MTLTLTVPPGAKVRVRITNENGETTEIEVEPGETVTITLPAGSTVR...,0.045301,0.863977,0.899356,1.202308,-0.716783,1.670006,2.163462,0.140161,...,1.068166,-0.102552,0.307800,0.091446,0.089552,1.860268,1.955047,hall,0,G11
980,2MFI.pdb,SIVRGVVVAIDKDVVLVDAGLKSESAIPAEQFKNAQGELEIQVGDE...,0.043516,0.653008,0.871242,0.843268,0.418390,1.332651,1.645558,0.057752,...,0.040325,0.154431,0.101648,0.068990,0.173077,1.499289,1.309840,220,0,G11
981,2KZI.pdb_pross8,NKFNKEMQNAYYEILHLPNLNEEQKNAFIQSLYDDPSQSANLLAEA...,0.003071,0.819099,0.836184,0.847138,-0.031946,2.039985,1.909280,0.000000,...,1.339580,1.454494,0.000000,1.423216,0.000000,1.509847,1.684587,71,0,G11


In [14]:
import warnings
warnings.simplefilter('ignore')

In [15]:
df_wts[df_wts['name']=='1AOY.pdb']

,name,aa_seq,frac_NA,raw_corr,dg_corr,slope,y_intercept,width_KT,width_KC,wt_dg_std_max,...,wt_kt,wt_kc,wt_k50_diff_max,wt_d_from_line,frac_pos_with_hydrophobic_stabilzing_muts,cut_potential_k50_max,cut_potential_dg_max,WT_cluster,num_Cys,DMS_group
238,1AOY.pdb,QEELVKAFKALLKEEKFSSQGEIVAALQEQGFDNINQSKVSRMLTK...,0.043336,0.975758,0.972606,1.17318,-0.271201,2.182514,2.515355,0.058767,...,0.755897,-0.006231,0.097032,0.04219,0.0,0.252398,0.114816,95,1,G0


In [16]:
i = 295
df_test = pd.DataFrame(select_data_set(df_wts.iloc[i,0],df_wts.iloc[i,1])).T
df_test.columns=['frac_NA','raw_corr','dg_corr','slope','y_intercept','width_KT','width_KC','wt_dg_std_max','wt_k50_std_max','wt_dg_max','wt_dgt_med','wt_dgc_med','wt_dg_med','wt_kt','wt_kc','wt_k50_diff_max','wt_d_from_line','frac_pos_with_hydrophobic_stabilzing_muts','cut_potential_k50_max','cut_potential_dg_max','destabilized_pos','cys_stabilized_pos']
df_test.T

,0
frac_NA,0.042367
raw_corr,0.983495
dg_corr,0.97288
slope,0.924168
y_intercept,0.742089
width_KT,2.161183
width_KC,2.363039
wt_dg_std_max,0.045039
wt_k50_std_max,0.033228
wt_dg_max,2.729849


In [ ]:
out,err_wt1 = [],[]

for i in tqdm(range(len(df_wts))):
    try:
        out.append(select_data_set(df_wts.iloc[i,0],df_wts.iloc[i,1]))
        
    except:
        print(i)
        err_wt1.append(i)

In [ ]:
outdf=pd.DataFrame(out)
outdf.columns=['frac_NA','raw_corr','dg_corr','slope','y_intercept','width_KT','width_KC','wt_dg_std_max','wt_k50_std_max','wt_dg_max','wt_dgt_med','wt_dgc_med','wt_dg_med','wt_kt','wt_kc','wt_k50_diff_max','wt_d_from_line','frac_pos_with_hydrophobic_stabilzing_muts','cut_potential_k50_max','cut_potential_dg_max','destabilized_pos','cys_stabilized_pos']
outdf

In [ ]:
df_wts_sum = pd.concat([df_wts.iloc[:,:2],outdf],axis=1)
df_wts_sum